In [42]:
# import libraries
import gspread
import os
import pandas as pd
import numpy as np
from todoist_api_python.api import TodoistAPI
import datetime as dt

In [24]:
# -- paths & file name settings --
# code home folder
pCode = '/Volumes/GoogleDrive/My Drive/Code'
# google credentials
pCredential_google = os.path.join(pCode, 'google_api','credentials.json')
# todoist credentials
pCredential_todoist = os.path.join(pCode, 'todoist','todoist_api_token.txt')
# todoist sheet
folder_id = '1Dl2nTvkOVYXdV6LBBgL0lLSVQO_Z85c5'
sheet_filename = 'todoist_subtask_planner'
sheet_id = '1rr-OtLlfabI9Mgja2sk-nwDCtjvwTskkuxmQPiIB8RE'

In [55]:
# connect to services
sa = gspread.service_account(filename = pCredential_google)
# open sheet
sh = sa.open_by_key(sheet_id)

In [57]:
# -- get parent task content -- 
wks = sh.worksheet("parent_task") # go to worksheet
content_parent = wks.acell('B1').value
content_parent

'Event123'

In [30]:
# -- get todoist parent task info --
# connect todoist api
with open (pCredential_todoist) as f:
    todoist_api = f.readlines()
    todoist_api = todoist_api[0]

# get task dictionary by name
api = TodoistAPI(todoist_api)
try:
    todoistpull = api.get_tasks()
except Exception as error:
    print(error)

# put in dictionaries
task_dict = {}
for i in range(len(todoistpull)):
    task_dict[todoistpull[i].content] = todoistpull[i].id

# get parent id
id_parent = task_dict[content_parent]

# get parent task info
task_parent = api.get_task(task_id=id_parent)

# get date and time of parent task
datetime_parent = task_parent.due.datetime
datetime_parent = dt.datetime.strptime(datetime_parent, '%Y-%m-%dT%H:%M:%S')

In [83]:
# -- get subtasks info --
wks2 = sh.worksheet("subtask") # go to worksheet
df = pd.DataFrame(wks2.get_all_records())

In [84]:
# -- set up subtasks attributes -- 
df['datetime_parent'] = datetime_parent 
# get date off set
df['due_date'] = df.datetime_parent + pd.to_timedelta(df.days_from_parent, unit='D')


In [100]:
# due date time
df['due_datetime'] = ''
for t,i in zip(df.time_from_parent, range(len(df.time_from_parent))):
    if isinstance(t, int):
        df.loc[i, 'due_datetime'] = df.due_date[i] + pd.to_timedelta(t, unit='m')


In [105]:
# STARTS HERE: build tasks content
i = 0
pd.Series([df.task[i]])


AttributeError: 'str' object has no attribute 'values'

In [101]:
# concatenate subtask content to add parent event title


In [102]:
df

,task,days_from_parent,time_from_parent,duration,label,priority,datetime_parent,due_date,due_datetime
0,prepare event,0,-60,60,_Computer,1,2022-08-07 17:00:00,2022-08-07 17:00:00,2022-08-07 16:00:00
1,minutes,0,60,15,"_Computer, _Home",1,2022-08-07 17:00:00,2022-08-07 17:00:00,2022-08-07 18:00:00
2,send agenda,3,,,_Computer,3,2022-08-07 17:00:00,2022-08-10 17:00:00,
3,remind people,2,0,15,,2,2022-08-07 17:00:00,2022-08-09 17:00:00,2022-08-09 17:00:00


In [ ]:
# upload subtasks